# link of dataset
https://www.kaggle.com/datasets/osamaomargaweesh/egyptian-coins

In [2]:
import json
import os
import shutil
import cv2
from tqdm import tqdm

# Set the base directory to your new directory
base_directory = r"D:\Vortex\ROV\Phase 2\deep learning\coin_detection"

# Paths to COCO annotations and image directories
train_annotation_file = os.path.join(base_directory, 'train/_annotations.coco.json')
valid_annotation_file = os.path.join(base_directory, 'valid/_annotations.coco.json')
test_annotation_file = os.path.join(base_directory, 'test/_annotations.coco.json')

train_image_folder = os.path.join(base_directory, 'train/images')
valid_image_folder = os.path.join(base_directory, 'valid/images')
test_image_folder = os.path.join(base_directory, 'test/images')

output_base = os.path.join(base_directory, 'prepared_data')
output_train_image_folder = os.path.join(output_base, 'train/images')
output_valid_image_folder = os.path.join(output_base, 'valid/images')
output_test_image_folder = os.path.join(output_base, 'test/images')

output_train_label_folder = os.path.join(output_base, 'train/labels')
output_valid_label_folder = os.path.join(output_base, 'valid/labels')
output_test_label_folder = os.path.join(output_base, 'test/labels')

# Create necessary directories if they don't exist
os.makedirs(output_train_image_folder, exist_ok=True)
os.makedirs(output_valid_image_folder, exist_ok=True)
os.makedirs(output_test_image_folder, exist_ok=True)
os.makedirs(output_train_label_folder, exist_ok=True)
os.makedirs(output_valid_label_folder, exist_ok=True)
os.makedirs(output_test_label_folder, exist_ok=True)

def convert_coco_to_yolo(coco_annotation_file, image_folder, output_image_folder, output_label_folder):
    with open(coco_annotation_file, 'r') as f:
        coco_data = json.load(f)

    category_dict = {category['id']: category['name'] for category in coco_data['categories']}
    
    for image_info in tqdm(coco_data['images']):
        image_id = image_info['id']
        image_filename = image_info['file_name']
        image_path = os.path.join(image_folder, image_filename)
        
        img = cv2.imread(image_path)
        img_height, img_width, _ = img.shape
        
        annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] == image_id]

        shutil.copy(image_path, os.path.join(output_image_folder, image_filename))

        label_file_path = os.path.join(output_label_folder, image_filename.replace('.jpg', '.txt'))
        with open(label_file_path, 'w') as label_file:
            for annotation in annotations:
                class_id = annotation['category_id'] - 1
                x_min, y_min, width, height = annotation['bbox']

                x_center = (x_min + width / 2) / img_width
                y_center = (y_min + height / 2) / img_height
                normalized_width = width / img_width
                normalized_height = height / img_height

                label_file.write(f"{class_id} {x_center} {y_center} {normalized_width} {normalized_height}\n")

# Convert all data: train, valid, and test
convert_coco_to_yolo(train_annotation_file, train_image_folder, output_train_image_folder, output_train_label_folder)
convert_coco_to_yolo(valid_annotation_file, valid_image_folder, output_valid_image_folder, output_valid_label_folder)
convert_coco_to_yolo(test_annotation_file, test_image_folder, output_test_image_folder, output_test_label_folder)


100%|██████████| 4/4 [00:00<00:00, 157.96it/s]


In [1]:
from ultralytics import YOLO

# Train the model
model = YOLO("yolov8n.pt")  # You can change to another YOLOv8 model

# Train the model with the prepared data
model.train(
    data=r"D:\Vortex\ROV\Phase 2\deep learning\coin_data.yaml",  # Path to your dataset YAML
    epochs=50,  # Number of training epochs
    imgsz=416,  # Image size (input to the model)
    batch=4,  # Batch size
    project=r"D:\Vortex\ROV\SW-Training-Phase2-2025\Card_2",  # Directory where the weights will be saved
    name="coin_model"  # You can give a name to the folder (e.g., "coin_model") for organization
)


New https://pypi.org/project/ultralytics/8.3.27 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.15  Python-3.12.3 torch-2.3.1+cpu CPU (AMD Ryzen 5 7535HS with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=D:\Vortex\ROV\Phase 2\deep learning\coin_data.yaml, epochs=50, time=None, patience=100, batch=4, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=D:\Vortex\ROV\SW-Training-Phase2-2025\Card_2, name=coin_model, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_

train: Scanning D:\Vortex\ROV\Phase 2\deep learning\coin_detection\prepared_data\train\labels.cache... 86 images, 0 backgrounds, 0 corrupt: 100%|██████████| 86/86 [00:00<?, ?it/s]
val: Scanning D:\Vortex\ROV\Phase 2\deep learning\coin_detection\prepared_data\valid\labels.cache... 9 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9/9 [00:00<?, ?it/s]


Plotting labels to D:\Vortex\ROV\SW-Training-Phase2-2025\Card_2\coin_model\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 63 weight(decay=0.0), 70 weight(decay=0.0005), 69 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 416 train, 416 val
Using 0 dataloader workers
Logging results to D:\Vortex\ROV\SW-Training-Phase2-2025\Card_2\coin_model
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G      0.766      3.459     0.9824         16        416: 100%|██████████| 22/22 [00:14<00:00,  1.49it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.34s/it]

                   all          9         59      0.018      0.801      0.151      0.111


...

                   all          9         59      0.958      0.998      0.995      0.903



50 epochs completed in 0.198 hours.
Optimizer stripped from D:\Vortex\ROV\SW-Training-Phase2-2025\Card_2\coin_model\weights\last.pt, 5.6MB
Optimizer stripped from D:\Vortex\ROV\SW-Training-Phase2-2025\Card_2\coin_model\weights\best.pt, 5.6MB

Validating D:\Vortex\ROV\SW-Training-Phase2-2025\Card_2\coin_model\weights\best.pt...
Ultralytics 8.3.15  Python-3.12.3 torch-2.3.1+cpu CPU (AMD Ryzen 5 7535HS with Radeon Graphics)
Model summary (fused): 186 layers, 2,684,953 parameters, 0 gradients, 6.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.02it/s]


                   all          9         59      0.967      0.997      0.995      0.909
            half_pound          9         22      0.983          1      0.995      0.908
             one_pound          9         19       0.92          1      0.995      0.904
           quart_pound          9         18          1       0.99      0.995      0.913
Speed: 0.7ms preprocess, 56.5ms inference, 0.0ms loss, 3.5ms postprocess per image
Results saved to D:\Vortex\ROV\SW-Training-Phase2-2025\Card_2\coin_model


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000223738DE480>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

In [ ]:
import os
import cv2
from ultralytics import YOLO

# Load the trained YOLO model
model = YOLO(r"D:\Vortex\ROV\SW-Training-Phase2-2025\Card_2\coin_model\weights\best.pt")

# Define the coin values in EGP
coin_values = {"one_pound": 1.0, "half_pound": 0.5, "quart_pound": 0.25}

# Define class names for detected objects (replace these with actual class names from your model)
class_names = ['half_pound','one_pound', 'quart_pound']  # Example class names

# Function to perform detection and calculate total value
def detect_and_calculate_total(image_path):
    # Read the image using OpenCV
    img = cv2.imread(image_path)

    # Perform inference on the image
    results = model(img)  # Get the results from the YOLO model

    # Initialize the total value and object counts
    total_value = 0
    object_counts = {class_name: 0 for class_name in class_names}

    # Iterate over the detected objects and calculate total value
    for result in results[0].boxes.data:
        class_id = int(result[5])  # The class id of the detected object
        confidence = result[4]     # The confidence score of the detection

        # Map the class_id to class_name (use the appropriate index based on your model's classes)
        class_name = class_names[class_id]
        
        # If the detected class is a recognized coin, add its value to the total
        if class_name in coin_values:
            total_value += coin_values[class_name] * confidence  # Weight by confidence score
            object_counts[class_name] += 1  # Count the object

            # Draw bounding box and label
            x_min, y_min, x_max, y_max = map(int, result[:4])
            cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)  # Green box
            label = f"{class_name} ({confidence * 100:.2f}%)"
            cv2.putText(img, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)  # Blue text

    # Display the image with bounding boxes and labels
    cv2.imshow("Detected Image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return total_value, object_counts


# Directory containing test images
test_image_directory = r"D:\Vortex\ROV\Phase 2\deep learning\coin_detection\test\images"

# Loop through all the image files in the directory
total_value_all_images = 0
total_object_counts_all_images = {class_name: 0 for class_name in class_names}

for image_filename in os.listdir(test_image_directory):
    # Only process image files (you can adjust file extensions as needed)
    if image_filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(test_image_directory, image_filename)
        
        # Perform detection and calculate the total value and object counts
        total_value, object_counts = detect_and_calculate_total(image_path)
        
        # Update the total value for all images
        total_value_all_images += total_value
        
        # Update the object counts for all images
        for class_name in class_names:
            total_object_counts_all_images[class_name] += object_counts[class_name]
        
        # Print the total value detected in the current image
        print(f"Total value detected in {image_filename}: {total_value:.2f} EGP")
        for class_name, count in object_counts.items():
            print(f"  {class_name}: {count} detected")

# Print the total value and object counts across all images
print(f"\nTotal value detected in all images: {total_value_all_images:.2f} EGP")
for class_name, count in total_object_counts_all_images.items():
    print(f"Total {class_name} detected in all images: {count}")



0: 416x416 2 one_pounds, 3 half_pounds, 2 quart_pounds, 101.1ms
Speed: 6.0ms preprocess, 101.1ms inference, 4.5ms postprocess per image at shape (1, 3, 416, 416)
Total value detected in IMG-20231112-WA0004_jpg.rf.aebe44d33ebc5be34cb0e4c93c7cf2b8.jpg: 4.37 EGP
  half_pound: 2 detected
  one_pound: 3 detected
  quart_pound: 2 detected

0: 416x416 4 one_pounds, 3 half_pounds, 2 quart_pounds, 54.5ms
Speed: 2.0ms preprocess, 54.5ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 416)
Total value detected in IMG-20231112-WA0005_jpg.rf.dad45e6c08e1cb5d15aea66b6583fe8c.jpg: 4.76 EGP
  half_pound: 4 detected
  one_pound: 3 detected
  quart_pound: 2 detected

0: 416x416 2 one_pounds, 1 half_pound, 2 quart_pounds, 55.2ms
Speed: 2.2ms preprocess, 55.2ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)
Total value detected in IMG-20231112-WA0028_jpg.rf.972d7996d8d2d1a0e75cd07c77e09f28.jpg: 2.45 EGP
  half_pound: 2 detected
  one_pound: 1 detected
  quart_pound: 2 detect